In [ ]:
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import glob, os
from scipy.signal import savgol_filter
import numpy as np
import seaborn as sn

In [ ]:
# source_dir= 'G:/Innovations@HELLER/DN/KI/Temperaturkompensation/2021_Spindelwachstumskompensation_KI_HSU_SC63/repaired_Files/Data_Verarbeitet/peak_removed_shifted/median/'
# model_dir= 'G:/Innovations@HELLER/DN/KI/Temperaturkompensation/2021_Spindelwachstumskompensation_KI_HSU_SC63/repaired_Files/Data_Verarbeitet/peak_removed_shifted/median/model.p'
# preprocessor_dir= 'G:/Innovations@HELLER/DN/KI/Temperaturkompensation/2021_Spindelwachstumskompensation_KI_HSU_SC63/repaired_Files/Data_Verarbeitet/peak_removed_shifted/median/preprocessor/preprocessor.p'
# new_data_dir= 'G:/Innovations@HELLER/DN/KI/Temperaturkompensation/2021_Spindelwachstumskompensation_KI_HSU_SC63/Messungen DC100 H5000 M57002/csvs/smoothed'

In [ ]:
source_dir= 'G:/Innovations@HELLER/DN/KI/Temperaturkompensation/2021_Spindelwachstumskompensation_KI_HSU_SC63/Messungen DC100 H5000 M57002/csvs/smoothed/'
preprocessor_dir= 'G:/Innovations@HELLER/DN/KI/Temperaturkompensation/2021_Spindelwachstumskompensation_KI_HSU_SC63/Messungen DC100 H5000 M57002/csvs/smoothed/AI_Model/preprocessor/preprocessor.p'
model_dir= 'G:/Innovations@HELLER/DN/KI/Temperaturkompensation/2021_Spindelwachstumskompensation_KI_HSU_SC63/Messungen DC100 H5000 M57002/csvs/smoothed/AI_Model/model.p'

In [ ]:
def generateDataSource(signal_input= None, input_columns: list = [], output_length: int=1, signal_output= None, window= 1 , shift= 1):
    Signal_Length = signal_input.shape[0]
    num_samples= int((Signal_Length - window + 1 ) / shift)
    x = np.zeros(shape=(num_samples, window, signal_input.shape[1]))
    y= np.zeros(shape=(num_samples, output_length, 1))#signal_output.shape[1]))
    for i in range (num_samples):
        x[i]= signal_input[i * shift : i * shift + window]
        y[i]= signal_output[i * shift + window - 1 : i * shift + window - 1+ output_length]
    return x, y

In [ ]:
from keras.models import Sequential
from keras.layers import Input, LSTM,Dense
from keras.losses import mean_absolute_error as mae, mean_squared_error as mse, mean_absolute_percentage_error as mape
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from joblib import load,dump
## Extract Data source from all signals
os.chdir(source_dir)
selected_Columns= ['t_bett','t_motor','t_spindel', 'drhz']#['t_spindel', 'drhz']#'DRHZ', 'M8', 'M21','M121', 'smoothed_T_Spindel']
results: np.ndarray= None
targets: np.ndarray= None
scaler= None
for file in glob.glob('*.csv'):
    if file == 'combined.csv':
        continue
    df= pd.read_csv(file)
    signals= df[selected_Columns]
    if scaler  is None:
        try:
            scaler= load(preprocessor_dir)
            print('preprocessor is loaded successfully')
        except FileNotFoundError as err:
            scaler= MinMaxScaler(feature_range=(-1,1))
            scaler= scaler.fit(signals)
            dump(scaler, preprocessor_dir)
            print('preprocessor is saved successfully')
    signals= scaler.transform(signals)
    signals= np.round(signals,2)
    output= (df['z_tcp_ok']).round(6)# df['Z_TCP_OK']#
    partitions, target= generateDataSource(signal_input= signals, input_columns= selected_Columns, output_length= 1,signal_output= output, window= 10, shift= 1)
    if results is None:
        results= partitions
        targets= target
    else:
        results= np.concatenate((results, partitions), axis= 0)
        targets= np.concatenate((targets, target), axis= 0)
    print('input: ', results.shape, ' output:', targets.shape)

In [ ]:
training_df= pd.read_csv(source_dir+'data for testing and training/training.csv')
testing_df= pd.read_csv(source_dir+'data for testing and training/test.csv')
print (training_df.info)
x_train= training_df['Predictors'].iloc(0)
y_train= training_df['z_tcp_ok'].iloc(0)
x_testing= testing_df['Predictors'].iloc(0)
y_testing= testing_df['z_tcp_ok'].iloc(0)
print (x_train)

In [ ]:
### Create Tolerance Lines
import numpy as np
tolerance= 0.005
x_steps= np.linspace(-0.03, 0.06, 10)
lowerline= x_steps - tolerance
upperline= x_steps + tolerance

In [ ]:
### Create Tolerance Lines
import numpy as np
tolerance= 0.01
x_steps= np.linspace(-0.03, 0.06, 10)
lowerline_actual= x_steps - tolerance
upperline_actual= x_steps + tolerance

In [ ]:
### Load Model
from keras.models import load_model
model=  Sequential()
model= load_model(filepath= model_dir)
print(model.summary())
pred= model.predict(x_train)
from keras.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
print ('mse= ',mean_squared_error(y_train,pred), ' mae= ', mean_absolute_error(y_train,pred), ' mape= ', mean_absolute_percentage_error(y_train,pred))
fig, ax= plt.subplots(figsize=(15,15))

plt.title('Results of RandomForest Algorithm')
plt.xlabel('True')
plt.ylabel('Predicted')
plt.grid()
plt.plot(x_steps, upperline, label= 'Upper Bound', color='green')
plt.scatter(targets,pred, label= 'predictions')
plt.plot(x_steps, lowerline, label= 'lower Bound', color='green')
plt.plot(x_steps, x_steps, label= 'Optimal line')
plt.plot(x_steps, lowerline_actual, label= 'Actual lower Bound', color='red')
plt.plot(x_steps, upperline_actual, label= 'Actual Upper Bound', color='red')
plt.legend()
plt.show()
